In [1]:
!pip install -q sagemaker sagemaker[local]

You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/hunkim-kogpt2'

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='hunkimSagemaker')['Role']['Arn']
    
print(role)

arn:aws:iam::294038372338:role/hunkimSagemaker


In [3]:
%%bash
mkdir data
touch data/README.md

mkdir: data: File exists


In [4]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-294038372338/sagemaker/hunkim-kogpt2


## Dummy train
It just store the models and voc data

In [10]:
from sagemaker.pytorch import PyTorch

m = PyTorch(entry_point='pytorch_train.py',
                    source_dir='KoGPT2',
                    role=role,
                    framework_version='1.5.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    #train_instance_type='local',

                    # available hyperparameters: emsize, nhid, nlayers, lr, clip, epochs, batch_size,
                    #                            bptt, dropout, tied, seed, log_interval
                    hyperparameters={
                        'epochs': 1,
                        'tied': True
                    })

In [11]:
m.fit({'training': inputs})

2020-06-09 11:28:06 Starting - Starting the training job...
2020-06-09 11:28:08 Starting - Launching requested ML instances......
2020-06-09 11:29:16 Starting - Preparing the instances for training......
2020-06-09 11:30:25 Downloading - Downloading input data...
2020-06-09 11:31:09 Training - Downloading the training image............
2020-06-09 11:33:24 Uploading - Uploading generated training modelbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-06-09 11:33:20,036 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-06-09 11:33:20,061 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-06-09 11:33:21,486 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-06-09 11:33:21,841 sagemaker-containers INFO     Installing module with the following command:
/opt/conda/bin/python -m pip install . -r requirements.txt
Pr

UnexpectedStatusException: Error for Training job pytorch-training-2020-06-09-11-27-58-220: Failed. Reason: AlgorithmError: InstallModuleError:
Command "/opt/conda/bin/python -m pip install . -r requirements.txt"

In [ ]:
import sagemaker
from sagemaker.mxnet.model import MXNetPredictor

def infer(input_sentence='아기 공룡 둘리는 희동이와', debug=False):
  predictor = MXNetPredictor(endpoint_name)

  pred_latency_sum = 0
  pred_count_sum = 0
  pred_cnt = 0

  for i in range(20):
    try:
      pred_out = predictor.predict(input_sentence)
      if i == 0:
        continue
      
      predicted_sentence= pred_out[0]
      predict_count = pred_out[1]
      predict_latency = pred_out[2]
    
      pred_latency_sum += predict_latency
      pred_count_sum =+ predict_count
      pred_cnt += 1
    except:
      print('Error and ingore it.')

  avg_latency = pred_latency_sum / pred_cnt
  avg_latency_per_inf = avg_latency / pred_count_sum

  if debug:
    print('Input sentence: {}'.format(input_sentence))
    print('Predicted sentence: {}'.format(predicted_sentence))
    print('Average number of inferenced token: {:.2f}'.format(pred_count_sum))
    print('Average inference latency for a sentence completion: {:.2f}'.format(avg_latency))
    print('Average inference latency per a token: {:.2f}\n'.format(avg_latency_per_inf))
  
  return predicted_sentence

In [ ]:
%%time
infer()

## Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [ ]:
predictor.delete_endpoint()